Copyright 2021-2022 Lawrence Livermore National Security, LLC and other MuyGPyS
Project Developers. See the top-level COPYRIGHT file for details.

SPDX-License-Identifier: MIT

# Univariate Regression Tutorial

This notebook walks through a simple regression workflow and explains the componenets of `MuyGPyS`.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# This is necessary if JAX is installed as the benchmark GP is not designed with JAX in mind.
from MuyGPyS import config
if config.muygpys_jax_enabled is True:
    config.update("muygpys_jax_enabled", False)
if config.muygpys_torch_enabled is True:
    config.update("muygpys_torch_enabled", False)

from MuyGPyS._test.gp import benchmark_sample, benchmark_sample_full, BenchmarkGP

We will set a random seed here for consistency when building docs.
In practice we would not fix a seed.

In [ ]:
np.random.seed(0)

## Sampling a Curve from a Conventional GP

This notebook will use a simple one-dimensional curve sampled from a conventional Gaussian process.
The Gaussian process in question is implemented in `MuyGPyS.testing.gp.BenchmarkGP`. 
It is intended for testing purposes only. 
We will specify the domain as a grid on a one-dimensional surface and divide the observations into train and test data.
Feel free to experiment with different parameters.

In [ ]:
lb = -10.0
ub = 10.0
data_count = 5001
train_step = 10
x = np.linspace(lb, ub, data_count).reshape(data_count, 1)
test_features = x[np.mod(np.arange(data_count), train_step) != 0, :]
train_features = x[::train_step, :]
test_count, _ = test_features.shape
train_count, _ = train_features.shape

We will now define a Matern kernel function and create a conventional GP.
Note here that we are setting a small value for `eps`, the prior variance of the "nugget" or diagonal noise matrix added to the raw kernel matrix to a small value purely for numerical stability.
This is an idealized experiment with no instrument error.

In [ ]:
nugget_var = 1e-14
fixed_length_scale = 1.0
benchmark_kwargs = {
    "kern": "matern",
    "metric": "l2",
    "eps": {"val": nugget_var},
    "nu": {"val": 2.0},
    "length_scale": {"val": fixed_length_scale},
}
gp = BenchmarkGP(**benchmark_kwargs)

Finally, we will sample a curve from this GP prior and visualize it.
Note that we perturb the train responses (the values that our model will actual receive) with Gaussian measurement noise.
Further note that this is not especially fast, as sampling from a conventional Gaussian process requires computing the Cholesky decomposition of a `(data_count, data_count)` matrix.

In [ ]:
y = benchmark_sample(gp, x)

In [ ]:
test_responses = y[np.mod(np.arange(data_count), train_step) != 0, :]
measurement_eps = 1e-5
train_responses = y[::train_step, :] + np.random.normal(0, measurement_eps, size=(train_count,1))

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 11))

axes[0].set_title("Sampled Curve", fontsize=24)
axes[0].set_xlabel("Feature Domain", fontsize=20)
axes[0].set_ylabel("Response Range", fontsize=20)
axes[0].plot(train_features, train_responses, "k*", label="perturbed train response")
axes[0].plot(test_features, test_responses, "g-", label="test response")
axes[0].legend(fontsize=20) 

vis_subset_size = 10
mid = int(train_count / 2)

axes[1].set_title("Sampled Curve (subset)", fontsize=24)
axes[1].set_xlabel("Feature Domain", fontsize=20)
axes[1].set_ylabel("Response Range", fontsize=20)
axes[1].plot(
    train_features[mid:mid + vis_subset_size], 
    train_responses[mid:mid + vis_subset_size], 
    "k*", label="perturbed train response"
)
axes[1].plot(
    test_features[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))], 
    test_responses[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))], 
    "g-", label="test response"
)

plt.tight_layout()

plt.show()

We will now attempt to recover the response on the held-out test data by training a univariate `MuyGPS` model on the perturbed training data.

## Constructing Nearest Neighbor Lookups

[NN_Wrapper](../MuyGPyS/neighbors.rst) 
is an api for tasking several KNN libraries with the construction of lookup indexes that empower fast training and inference. 
The wrapper constructor expects the training features, the number of nearest neighbors, and a method string specifying which algorithm to use, as well as any additional kwargs used by the methods. 
Currently supported implementations include 
[exact KNN using sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) ("exact") and 
[approximate KNN using hnsw](https://github.com/nmslib/hnswlib) ("hnsw", requires installing `MuyGPyS` using the `hnswlib` extras flag).

Here we construct an exact KNN data example with k = 30

In [ ]:
from MuyGPyS.neighbors import NN_Wrapper
nn_count = 30
nbrs_lookup = NN_Wrapper(train_features, nn_count, nn_method="exact", algorithm="ball_tree")

This `nbrs_lookup` index is then usable to find the nearest neighbors of queries in the training data.

## Sampling Batches of Data

`MuyGPyS` includes convenience functions for sampling batches of data from existing datasets.
These batches are returned in the form of row indices, both of the sampled data as well as their nearest neighbors.

Here we sample a random batch of `train_count` elements. 
This results in using *all* of the train data for training. 
We only do that in this case because this example uses a relatively small amount of data.
In practice, we would instead set `batch_count` to a resaonable number.
In practice we find reasonable values to be in the range of 500-2000.

In [ ]:
from MuyGPyS.optimize.batch import sample_batch
batch_count = train_count
batch_indices, batch_nn_indices = sample_batch(
    nbrs_lookup, batch_count, train_count
)

These `indices` and `nn_indices` arrays are the basic operating blocks of `MuyGPyS` linear algebraic inference.
The elements of `indices.shape == (batch_count,)` lists all of the row indices into `train_features` and `train_responses` corresponding to the sampled data.
The rows of `nn_indices.shape == (batch_count, nn_count)` list the row indices into `train_features` and `train_responses` corresponding to the nearest neighbors of the sampled data.

While the user need not use the 
[MuyGPyS.optimize.batch](../MuyGPyS/optimize/batch.rst) 
sampling tools to construct these data, they will need to construct similar indices into their data in order to use `MuyGPyS`.

## Setting and Optimizing Hyperparameters

One initializes a 
[MuyGPS](../MuyGPyS/gp/MuyGPS.rst)
object by indicating the kernel, as well as optionally specifying hyperparameters.

Consider the following example, which constructs a Matern kernel with all parameters fixed aside from `"nu"` and `"sigma_sq"`.
Note that we are here making the simplifying assumtions that we know the true `length_scale` and `measurement error`. 

In [ ]:
from MuyGPyS.gp.muygps import MuyGPS
k_kwargs = {
    "kern": "matern",
    "metric": "l2",
    "eps": {"val": measurement_eps},
    "nu": {"val": "log_sample", "bounds": (0.1, 5.0)},
    "length_scale": {"val": fixed_length_scale},
}
muygps = MuyGPS(**k_kwargs)

Hyperparameters can be initialized or reset using dictionary arguments containing the optional `"val"` and `"bounds"` keys.
`"val"` sets the hyperparameter to the given value, and `"bounds"` determines the upper and lower bounds to be used for optimization.
If `"bounds"` is set, `"val"` can also take the arguments `"sample"` and `"log_sample"` to generate a uniform or log uniform sample, respectively.
If `"bounds"` is set to `"fixed"`, the hyperparameter will remain fixed during any optimization.
This is the default behavior for all hyperparameters if `"bounds"` is unset by the user.

One sets the model hyperparameter `eps`, as well as kernel-specific hyperparameters, e.g. `nu` and  `length_scale` for the Matern kernel, at initialization as above.

There is one common hyperparameter, the `sigma_sq` scale parameter, that violates these rules.
`sigma_sq` cannot be directly set by the user, and always initializes to the value `"unlearned"`. 
We will show how to train `sigma_sq` below.
All hyperparameters other than `sigma_sq` are assumed to be fixed unless otherwise specified.

MuyGPyS depends upon linear operations on specially-constructed tensors in order to efficiently estimate GP realizations.
Constructing these tensors depends upon the nearest neighbor index matrices that we described above.
We can construct a distance tensor coalescing all of the square pairwise distance matrices of the nearest neighbors of a batch of points.

This snippet constructs a matrix of shape `(batch_count, nn_count)` coalescing all of the distance vectors between the same batch of points and their nearest neighbors.

In [ ]:
from MuyGPyS.gp.distance import crosswise_distances
batch_crosswise_dists = crosswise_distances(
    train_features, 
    train_features, 
    batch_indices,
    batch_nn_indices,
    metric="l2",
)

We can similarly construct a Euclidean distance tensor of shape `(batch_count, nn_count, nn_count)` containing the pairwise distances of the nearest neighbor sets of each sampled batch element.

In [ ]:
from MuyGPyS.gp.distance import pairwise_distances
pairwise_dists = pairwise_distances(
    train_features, batch_nn_indices, metric="l2"
)

The `MuyGPS` object we created earlier allows us to easily realize kernel tensors by way of its kernel function.

In [ ]:
Kcross = muygps.kernel(batch_crosswise_dists)
K = muygps.kernel(pairwise_dists)

In order to perform Gaussian process regression, we must utilize these kernel tensors in conjunction with their associated known responses.
We can construct these matrices using the index matrices we derived earlier.

In [ ]:
batch_targets = train_responses[batch_indices, :]
batch_nn_targets = train_responses[batch_nn_indices, :]

Since we often must realize `batch_targets` and `batch_nn_targets` in close proximity to `batch_crosswise_dists` and `batch_pairwise_dists`, the library includes a convenience function 
[`make_train_tensors()`](../MuyGPyS/gp/distance.rst)
that bundles these operations.

In [ ]:
from MuyGPyS.gp.distance import make_train_tensors
(
    batch_crosswise_dists,
    batch_pairwise_dists,
    batch_targets,
    batch_nn_targets,
) = make_train_tensors(
    muygps.kernel.metric,
    batch_indices,
    batch_nn_indices,
    train_features,
    train_responses,
)

We supply a convenient leave-one-out cross-validation utility 
([`optimize_from_tensors()`](../MuyGPyS/gp/distance.rst))
that utilizes these tensors to repeatedly realize kernel tensors during optimization.
This optimization loop wraps a few different batch optimization methods:
* [`scipy.optimize.minimize`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.minimize.html) - specifically uses the "L-BFGS-B" algorithm.
* [`bayes_opt.BayesianOptimization`](https://github.com/fmfn/BayesianOptimization) - the `optimize_from_tensors` wrapper only supports batch mode; examine the internals of the function if you would like to use Bayesian optimization interactively.

If we want to use `scipy`-style optimization, we pass the `opt_method="scipy"` kwarg.
While it is possible to pass additional kwargs based upon 
[`scipy.optimize.minimize`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.minimize.html),
it is presently unlikely that a user would want to do so.

In [ ]:
from MuyGPyS.optimize.chassis import optimize_from_tensors
muygps_scipy = optimize_from_tensors(
    muygps,
    batch_targets,
    batch_nn_targets,
    batch_crosswise_dists,
    batch_pairwise_dists,
    loss_method="mse",
    obj_method="loo_crossval",
    opt_method="scipy",
    verbose=True,
)

Similarly, we can use `bayesian-optimization` style optimization by passing the
`opt_method="bayesian"` (alternately `"bayes"` or `"bayes_opt"`) kwarg.
There are several additional parameters that a user might want to set.
In particular, `init_points` (the number of "exploration" objective function evaluations to perform)
and `n_iter` (the number of "exploitation" objective function evaluations to perform) are of use to most users.
This example also sets `random_state` for consistency.
See the documentation of [BayesianOptimization](https://github.com/fmfn/BayesianOptimization) for more examples.

In [ ]:
muygps_bayes = optimize_from_tensors(
    muygps,
    batch_targets,
    batch_nn_targets,
    batch_crosswise_dists,
    batch_pairwise_dists,
    loss_method="mse",
    obj_method="loo_crossval",
    opt_method="bayesian",
    verbose=True,
    random_state=1,
    init_points=5,
    n_iter=20,
)

In [ ]:
print(f"scipy.optimize.opt.minimize finds that the optimimal `nu` is {muygps_scipy.kernel.nu()}")
print(f"BayesianOptimization finds that the optimimal `nu` is {muygps_bayes.kernel.nu()}")

Note here that the returned value for `nu` might be different from the `nu` used by the conventional GP.

If you do not need to retain the distances tensors or batch targets for future reference, you can use a related function that realizes them internally.

In [ ]:
# from MuyGPyS.optimize.chassis import optimize_from_indices
# muygps = optimize_from_indices(
#     muygps,
#     batch_indices,
#     batch_nn_indices,
#     train_features,
#     train_responses,
#     loss_method="mse",
#     obj_method="loo_crossval",
#     opt_method="scipy",
#     verbose=False,
# )

As it is a variance scaling parameter that is insensitive to prediction-based optimization, we separately optimize `sigma_sq`.
In this case, we invoke 
[muygps_sigma_sq_optim()](../MuyGPyS/optimize/sigma_sq.rst), 
which approximates `sigma_sq` based upon the mean of the closed-form `sigma_sq` solutions associated with each of its batched nearest neighbor sets.
Note that this method is sensitive to several factors, include `batch_count`, `nn_count`, and the overall size of the dataset, tending to perform better as each of these factors increases. 

This is usually performed after optimizing other hyperparameters.

In [ ]:
from MuyGPyS.optimize.sigma_sq import muygps_sigma_sq_optim

scipy_K = muygps_scipy.kernel(batch_pairwise_dists)
muygps_scipy = muygps_sigma_sq_optim(muygps_scipy, batch_pairwise_dists, batch_nn_targets, sigma_method="analytic")
bayes_K = muygps_bayes.kernel(batch_pairwise_dists)
muygps_bayes = muygps_sigma_sq_optim(muygps_bayes, batch_pairwise_dists, batch_nn_targets, sigma_method="analytic")
print(f"scipy-optimized sigma_sq: {muygps_scipy.sigma_sq()}")
print(f"BayesianOptimization-optimized sigma_sq: {muygps_bayes.sigma_sq()}")

## Inference

With set (or learned) hyperparameters, we are able to use the `muygps` object to predict the response of test data.
Several workflows are supported.

See below a simple regression workflow, using the data structures built up in this example.
This workflow uses the compact tensor-making function 
[make_regress_tensors()](../MuyGPyS/gp/distance.rst)
to succinctly create tensors defining the `pairwise_dists` among each nearest neighbor set, the `crosswise_dists` between each test point and its nearest neighbor set, and the `nn_targets` or responses of the nearest neighbors in each set.
We then create the `Kcross` cross-covariance matrix and `K` covariance tensor and pass them to [MuyGPS.regress()](../MuyGPyS/gp/MuyGPS.rst) in order to obtain our predictions.

In [ ]:
from MuyGPyS.gp.distance import make_regress_tensors

# make the indices
test_count, _ = test_features.shape
indices = np.arange(test_count)
nn_indices, _ = nbrs_lookup.get_nns(test_features)

# make distance and target tensors
(
    crosswise_dists,
    pairwise_dists,
    nn_targets,
) = make_regress_tensors(
    muygps.kernel.metric,
    indices,
    nn_indices,
    test_features,
    train_features,
    train_responses,
)

# Make the kernels
scipy_Kcross = muygps_scipy.kernel(crosswise_dists)
scipy_K = muygps_scipy.kernel(pairwise_dists)

bayes_Kcross = muygps_bayes.kernel(crosswise_dists)
bayes_K = muygps_bayes.kernel(pairwise_dists)

# perform Gaussian process regression 
scipy_predictions, scipy_variances = muygps_scipy.regress(
    scipy_K,
    scipy_Kcross,
    train_responses[nn_indices, :],
    variance_mode="diagonal",
    apply_sigma_sq=True,
)

bayes_predictions, bayes_variances = muygps_bayes.regress(
    bayes_K,
    bayes_Kcross,
    train_responses[nn_indices, :],
    variance_mode="diagonal",
    apply_sigma_sq=True,
)

We here evaluate our prediction performance in terms of RMSE, mean diagonal posterior variance, the mean 95% confidence interval size, and the coverage, which ideally should be near 95%. 

In [ ]:
from MuyGPyS.optimize.loss import mse_fn

scipy_confidence_intervals = np.sqrt(scipy_variances) * 1.96
scipy_coverage = (
    np.count_nonzero(
        np.abs(test_responses - scipy_predictions)
        < scipy_confidence_intervals.reshape(test_count, 1)
    )
    / test_count
)
bayes_confidence_intervals = np.sqrt(bayes_variances) * 1.96
bayes_coverage = (
    np.count_nonzero(
        np.abs(test_responses - bayes_predictions)
        < bayes_confidence_intervals.reshape(test_count, 1)
    )
    / test_count
)
print(f"scipy results:")
print(f"\tRMSE: {np.sqrt(mse_fn(scipy_predictions, test_responses))}")
print(f"\tmean diagonal variance: {np.mean(scipy_variances)}")
print(f"\tmean confidence interval size: {np.mean(scipy_confidence_intervals * 2)}")
print(f"\tcoverage: {scipy_coverage}")

print(f"bayes results:")
print(f"\tRMSE: {np.sqrt(mse_fn(bayes_predictions, test_responses))}")
print(f"\tmean diagonal variance: {np.mean(bayes_variances)}")
print(f"\tmean confidence interval size: {np.mean(bayes_confidence_intervals * 2)}")
print(f"\tcoverage: {bayes_coverage}")

Again, if you do not need to reuse your tensors, you can run the more compact workflow:

In [ ]:
# # make the indices
# test_count, _ = test_features.shape
# indices = np.arange(test_count)
# nn_indices, _ = nbrs_lookup.get_nns(test_features)

# scipy_predictions, scipy_variance = scipy_muygps.regress_from_indices(
#     indices, 
#     nn_indices,
#     test_features,
#     train_features,
#     train_responses,
#     variance_mode="diagonal",
#     apply_sigma_sq=True,
# )

These regression examples return predictions (posterior means) and variances for each element of the test dataset.
These variances are in the form of diagonal and independent variances that encode the uncertaintainty of the model's predictions at each test point.
To scale the variances, they should be multiplied by the trained `sigma_sq` scaling parameters, of which there will be one scalar associated with each dimension of the response.
The kwarg `apply_sigma_sq=True` indicates that this scaling will be performed automatically.
This is the default behavior, but will be skipped if `sigma_sq == "unlearned"`.

For a univariate resonse whose variance is obtained with `apply_sigma_sq=False`, the scaled predicted variance is equivalent to multiplying the predicted variances by `muygps.sigma_sq()`.

In [ ]:
# scipy_predictions, scipy_variances = scipy_muygps.regress_from_indices(
#     indices, 
#     nn_indices,
#     test_features,
#     train_features,
#     train_responses,
#     variance_mode="diagonal",
#     apply_sigma_sq=False,
# )
# scipy_scaled_variance = scipy_muygps.sigma_sq() * scipy_variances

We can also plot our responses and evaluate their performance. 
We plot below the predicted and true curves, as well as the 95% confidence interval.
We plot a smaller subset of the data in the lower curve in order to better scrutinize the 95% confidence interval.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 16))

axes[0].set_title("Sampled Curve with scipy optimized model", fontsize=24)
axes[0].set_xlabel("Feature Domain", fontsize=20)
axes[0].set_ylabel("Response Range", fontsize=20)
axes[0].plot(train_features, train_responses, "k*", label="perturbed train response")
axes[0].plot(test_features, test_responses, "g-", label="test response")
axes[0].plot(test_features, scipy_predictions, "r--", label="test predictions")
axes[0].fill_between(
    test_features[:, 0], 
    (scipy_predictions[:, 0] - scipy_confidence_intervals),
    (scipy_predictions[:, 0] + scipy_confidence_intervals),
    facecolor="red",
    alpha=0.25,
    label="95% Confidence Interval",
)
axes[0].legend(fontsize=20)

axes[1].set_title("Sampled Curve (subset) with scipy optimized model", fontsize=24)
axes[1].set_xlabel("Feature Domain", fontsize=20)
axes[1].set_ylabel("Response Range", fontsize=20)
axes[1].plot(
    train_features[mid:mid + vis_subset_size], 
    train_responses[mid:mid + vis_subset_size], 
    "k*", label="perturbed train response"
)
axes[1].plot(
    test_features[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))], 
    test_responses[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))], 
    "g-", label="test response"
)
axes[1].plot(
    test_features[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))], 
    scipy_predictions[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))],
    "r--", label="test predictions")
axes[1].fill_between(
    test_features[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))][:, 0], 
    (scipy_predictions[:, 0] - scipy_confidence_intervals)[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))],
    (scipy_predictions[:, 0] + scipy_confidence_intervals)[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))],
    facecolor="red",
    alpha=0.25,
    label="95% Confidence Interval",
)
axes[1].legend(fontsize=20)

axes[2].set_title("Sampled Curve (subset) with Bayesian optimized model", fontsize=24)
axes[2].set_xlabel("Feature Domain", fontsize=20)
axes[2].set_ylabel("Response Range", fontsize=20)
axes[2].plot(
    train_features[mid:mid + vis_subset_size], 
    train_responses[mid:mid + vis_subset_size], 
    "k*", label="perturbed train response"
)
axes[2].plot(
    test_features[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))], 
    test_responses[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))], 
    "g-", label="test response"
)
axes[2].plot(
    test_features[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))], 
    bayes_predictions[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))],
    "b--", label="test predictions")
axes[2].fill_between(
    test_features[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))][:, 0], 
    (bayes_predictions[:, 0] - bayes_confidence_intervals)[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))],
    (bayes_predictions[:, 0] + bayes_confidence_intervals)[mid * (train_step - 1):mid * (train_step - 1) + (vis_subset_size * (train_step - 1))],
    facecolor="blue",
    alpha=0.25,
    label="95% Confidence Interval",
)
axes[2].legend(fontsize=20)


plt.tight_layout()

plt.show()